In [1]:
import datasurfer as ds
from datasurfer.lib_llm.utility import agent_tool
from datasurfer.lib_llm.tool import read_file, write_file, web_search, get_current_datetime, list_directory_contents, \
                                    parse_rss_feed
from IPython.display import display, Markdown

https://openai.github.io/openai-agents-python/
https://google.github.io/adk-docs/


In [2]:
@agent_tool
def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the current weather and return a summary.
    location: The city and state, e.g. San Francisco, CA
    unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the current weather
    
    """
    return f"It is currently sunny in {location} and 75 degrees {unit}."

@agent_tool
def get_tomorrows_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the weather for tomorrow and return a summary.
    param location: The city and state, e.g. San Francisco, CA
    param unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the weather for tomorrow
    """
    return f"Tomorrow it will be rainy in {location} and 60 degrees {unit}."

In [3]:
tools = [get_current_weather, get_tomorrows_weather, read_file, write_file, list_directory_contents, parse_rss_feed]
mcp_servers = 'http://localhost:8000/mcp'

In [ ]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
len(await agent.get_available_tools())


9

In [5]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers, 
                 skills=r'C:\20_Develop\20_Resource\10_Github\skills\awesome-claude-skills')
print(agent.skills.to_text())

{
    "name": "artifacts-builder",
    "description": "Suite of tools for creating elaborate, multi-component claude.ai HTML artifacts using modern frontend web technologies (React, Tailwind CSS, shadcn/ui). Use for complex artifacts requiring state management, routing, or shadcn/ui components - not for simple single-file HTML/JSX artifacts.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "brand-guidelines",
    "description": "Applies Anthropic's official brand colors and typography to any sort of artifact that may benefit from having Anthropic's look-and-feel. Use it when brand colors or style guidelines, visual formatting, or company design standards apply.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "canvas-design",
    "description": "Create beautiful visual art in .png and .pdf documents using design philosophy. You should use this skill when the user asks to create a poster, piece of art, design, or other static piece. Create original visu

In [13]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
Markdown(await agent.resp('hi, what are your skills?', compression=True))

Assistant:

    Hi — here’s a quick summary of what I can do and the tool-assisted
    capabilities available in this chat.  General capabilities - Answer
    questions and explain concepts across many domains (science, history,
    finance, law basics, etc.). - Write and edit text: emails, reports, essays,
    proposals, resumes, marketing copy. - Code: generate, explain, debug, and
    refactor code in many languages (Python, JavaScript, Java, C#, SQL, etc.). -
    Math and data work: algebra, calculus, statistics, unit conversions, numeric
    reasoning. - Translate and summarize text; create outlines, checklists, and
    step-by-step plans. - Creative tasks: stories, poems, prompts,
    brainstorming, UX/copy suggestions. - Multimodal: analyze and describe
    images you upload, extract text from images, and answer questions about
    visuals. - File help: read and produce plain-text outputs; help with
    document content and formatting guidance.  Tool-assisted capabilities
    av

Hi — here’s a quick summary of what I can do and the tool-assisted capabilities available in this chat.

General capabilities
- Answer questions and explain concepts across many domains (science, history, finance, law basics, etc.).
- Write and edit text: emails, reports, essays, proposals, resumes, marketing copy.
- Code: generate, explain, debug, and refactor code in many languages (Python, JavaScript, Java, C#, SQL, etc.).
- Math and data work: algebra, calculus, statistics, unit conversions, numeric reasoning.
- Translate and summarize text; create outlines, checklists, and step-by-step plans.
- Creative tasks: stories, poems, prompts, brainstorming, UX/copy suggestions.
- Multimodal: analyze and describe images you upload, extract text from images, and answer questions about visuals.
- File help: read and produce plain-text outputs; help with document content and formatting guidance.

Tool-assisted capabilities available here
- calculate: evaluate mathematical expressions precisely.
- get_current_time: return current date/time.
- get_current_weather / get_tomorrows_weather: fetch weather summaries for a location.
- parse_rss_feed: read and summarize RSS feeds.
- read_file / write_file / list_directory_contents: inspect and edit files (text) on the environment I can access.
- read_skill_instruction / read_skill_reference / execute_skill_script: inspect and run predefined skill instructions or scripts when a task relates to those skills.
- unit_convert: convert simple units (m, cm, kg, g).
- multi_tool_use.parallel: run multiple compatible tools in parallel.

Notes and limitations
- I don’t have general unrestricted web browsing unless a specific tool provides it (RSS, weather, scripts). I can’t access personal accounts unless you provide content or grant access through supported means.
- For binary files like .docx, PDFs, or large datasets, I can extract and edit content if you upload them or request specific actions; some advanced document skills are supported via the skill framework when relevant.
- I can’t perform illegal or unsafe actions.

What would you like help with right now?

In [ ]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
Markdown(await agent.resp('What is the current date and time?', compression=True))

Assistant:

    Function Call:
        Name: get_current_time
        Arguments: {}
        Call ID: call_RDV9mvKyiKZe4EzBc4HZvNZI


------------------------------------------------------------------------------------

Assistant:

    The current date and time is 2026-01-12 08:46:54.

------------------------------------------------------------------------------------



The current date and time is 2026-01-12 08:46:54.

In [11]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('What is the weather like in Stuttgart Germany today and tomorrow?', compression=True))

Assistant:

    Function Call:
        Name: get_current_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_5tL5jnOtpeDL49mFSM4VBxHJ


    Function Call:
        Name: get_tomorrows_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_SSAO6VgN9JAQ7Gcoble0e8r8


------------------------------------------------------------------------------------

Assistant:

    Quick answer: the feed I checked says it’s sunny in Stuttgart today and
    rainy tomorrow. However—important note—the temperature values returned look
    wrong: the feed gave “75 degrees celsius” for today and “60 degrees celsius”
    for tomorrow, which is implausible.  Likely explanations: - The API
    mislabeled units. If those numbers were actually degrees Fahrenheit, they
    correspond to about 24°C today (75°F ≈ 23.9°C) and about 16°C tomorrow (60°F
    ≈ 15.6°C). - Or the API returned incorrect temperature data.  What would

Quick answer: the feed I checked says it’s sunny in Stuttgart today and rainy tomorrow. However—important note—the temperature values returned look wrong: the feed gave “75 degrees celsius” for today and “60 degrees celsius” for tomorrow, which is implausible.

Likely explanations:
- The API mislabeled units. If those numbers were actually degrees Fahrenheit, they correspond to about 24°C today (75°F ≈ 23.9°C) and about 16°C tomorrow (60°F ≈ 15.6°C).
- Or the API returned incorrect temperature data.

What would you like me to do next?
- Re-fetch the forecast (I can try again and request Fahrenheit or Celsius explicitly).
- Check a different weather source for confirmation and more details (precipitation chance, wind, hourly forecast).
- Use the Fahrenheit values converted to Celsius as the forecast I gave above.

In [ ]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers)
Markdown(await agent.resp('What is the weather like in Stuttgart Germany today and tomorrow?', compression=True))

In [23]:
# https://www.ft.com/rss/home/international
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.reset().resp(r'parse the rss feed from "C:\20_Develop\88_Training\71_LLM\skills\parse-rss\references", '
 'summarize the news, merge all of them in a markdown table with the format provided in instructions', compression=True))

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"parse-rss"}
        Call ID: call_VV2sKHBbgcOi4mhj0wIXkggh


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: list_directory_contents
        Arguments:
    {"directory_path":"C:\\20_Develop\\88_Training\\71_LLM\\skills\\parse-
    rss\\references"}
        Call ID: call_KvpOSqRjeLLG8fs6dYmYrCG6


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_file
        Arguments:
    {"file_path":"C:\\20_Develop\\88_Training\\71_LLM\\skills\\parse-
    rss\\references\\latest.rss"}
        Call ID: call_MBN6cazH06ohkSU26prhFZfT


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_file
        Arguments:
    {"file_path":"C:\\20_Develop\\88_Trainin

Summary:
Collected and parsed 17 items from two RSS files (latest.rss and news_25.rss). Content covers Indian domestic and world news (law-and-order, policy moves, public programs) and financial/market headlines (corporate actions, stocks, analyst moves).

| Title | Link | Publication Date |
|-------|------|------------------|
| Punjab orders major IPS cadre reshuffle, officers promoted across ranks | [Link](https://www.business-standard.com/india-news/punjab-govt-orders-major-reshuffle-promotions-in-ips-cadre-126011000921_1.html) | 2026-01-11 |
| Turkman Gate violence: Delhi court sends 3 more accused to judicial custody | [Link](https://www.business-standard.com/india-news/turkman-gate-violence-delhi-court-sends-3-more-accused-to-judicial-custody-126011000865_1.html) | 2026-01-10 |
| Iran warns protesters with death penalty, calls them 'enemies of God' | [Link](https://www.business-standard.com/world-news/iran-attorney-general-threatens-protesters-with-severe-punishment-126011000922_1.html) | 2026-01-10 |
| Scindia launches speed post 24 and 48, pledges faster India Post delivery | [Link](https://www.business-standard.com/india-news/scindia-launches-speed-post-24-and-48-pledges-faster-india-post-delivery-126011000871_1.html) | 2026-01-10 |
| SIR in MP: Voter list update reunites man with mother after 22 years | [Link](https://www.business-standard.com/india-news/mp-sir-leads-to-man-reunion-with-mother-after-22-years-126011000846_1.html) | 2026-01-10 |
| Punjab CM aims to turn 2nd phase of anti-drug drive into mass movement | [Link](https://www.business-standard.com/india-news/aap-second-phase-anti-drug-campaign-mass-movement-punjab-cm-126011000855_1.html) | 2026-01-10 |
| Trump proposes 1-year, 10% cap on credit card rates, banks push back | [Link](https://www.business-standard.com/world-news/trump-pushes-1-year-10-per-cent-cap-credit-card-interest-rates-126011000856_1.html) | 2026-01-10 |
| Hilton drops Minneapolis hotel over cancelled ICE bookings | [Link](https://www.investing.com/news/stock-market-news/hilton-removes-minneapolis-hotel-that-cancelled-reservation-to-immigration-agents-4432900) | 2026-01-06 |
| Brazil stocks higher at close of trade; Bovespa up 1.00% | [Link](https://www.investing.com/news/stock-market-news/brazil-stocks-higher-at-close-of-trade-bovespa-up-100-4433389) | 2026-01-06 |
| Credit bureau stocks tumble as FHFA chief criticizes pricing practices | [Link](https://www.investing.com/news/stock-market-news/credit-bureau-stocks-tumble-as-fhfa-chief-criticizes-pricing-practices-93CH-4433387) | 2026-01-06 |
| Top Canadian Energy Stocks for 2026: WarrenAI Analyzes BMO Picks | [Link](https://www.investing.com/news/stock-market-news/top-canadian-energy-stocks-for-2026-warrenai-analyzes-bmo-picks-93CH-4433363) | 2026-01-06 |
| Trump administration to advise Americans to limit sugar to 10 grams per meal - Bloomberg | [Link](https://www.investing.com/news/stock-market-news/trump-administration-to-advise-americans-to-limit-sugar-to-10-grams-per-meal--bloomberg-93CH-4433360) | 2026-01-06 |
| Meta names Microsoft’s Mahoney as chief legal officer | [Link](https://www.investing.com/news/stock-market-news/meta-names-microsofts-mahoney-as-chief-legal-officer-4433334) | 2026-01-06 |
| SoFi stock tumbles after BofA maintains underperform rating | [Link](https://www.investing.com/news/stock-market-news/sofi-stock-tumbles-after-bofa-maintains-underperform-rating-93CH-4433328) | 2026-01-06 |
| Comerica shareholders approve $10.9 billion deal with Fifth Third Bancorp | [Link](https://www.investing.com/news/stock-market-news/exclusivecomerica-shareholders-approve-109-billion-deal-with-fifth-third-bancorp-4433249) | 2026-01-06 |
| Marvell stock falls after announcing XConn Technologies acquisition | [Link](https://www.investing.com/news/stock-market-news/marvell-stock-falls-after-announcing-xconn-technologies-acquisition-93CH-4433319) | 2026-01-06 |
| Roblox stock falls after TD Cowen cuts price target on growth concerns | [Link](https://www.investing.com/news/stock-market-news/roblox-stock-falls-after-td-cowen-cuts-price-target-on-growth-concerns-93CH-4433317) | 2026-01-06 |

If you want:
- the table sorted differently (by date asc/desc), or
- additional columns (source, author, short description), or
- the output in CSV/Excel,
tell me which and I will produce it.

In [ ]:
Markdown(await agent.resp('Web search sport news from Stuttgart of the current month. Return at least 5 news. ' 
' write the results to a file called "stuttgart_sport_news.md"', compression=True))   

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"web-search"}
        Call ID: call_zBxuZrONdZKaqJXWBqBUHg1o


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_skill_reference
        Arguments: {"skill_name":"web-
    search","reference_link":"references\\script_usage_guide.md"}
        Call ID: call_PLGlgUsOzBjCk9e2GVWtql12


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts\\web_search.py"}
        Call ID: call_CPX9bQLSChsLJihP8D5AVEcM


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: get_current_time
        Arguments: {}
        Call ID: call_mUdw1Em1c9FMndsnU6bFoeBl


-------------------

I searched the web for Stuttgart sport news for the current month and saved at least five items to stuttgart_sport_news.md.

File created: stuttgart_sport_news.md
Summary of included items (all January 2026):
- Stuttgart dismantles Leverkusen in emphatic away Bundesliga display — Morung Express (11 Jan)
- Stuttgart end Leverkusen rot with big 4-1 win; Heidenheim, Mainz draw — Daily Finland (11 Jan)
- Football: German League returns — preview/coverage including Stuttgart — Sports GR8 (9 Jan)
- Live commentary: Leverkusen vs Stuttgart — Sky Sports (10 Jan)
- Leverkusen vs. Stuttgart live score & report — ESPN (10 Jan)
- VfB Stuttgart end seven-year hoodoo to kick start 2026 — Bundesliga.com
- 1:4-Heimpleite von Leverkusen gegen Stuttgart — Südtirol News (10 Jan)
- Wettersorgen: VfB reist lieber frühzeitig nach Leverkusen — t-online.de (8 Jan)
- Stuttgart thrash Bayer in Leverkusen before halftime — Zamin.uz

If you want:
- I can open and display the file contents here.
- Add more items, restrict to Stuttgart city teams beyond VfB (e.g., local handball, ice hockey), or fetch full article texts.

In [7]:
agent.history.to_dataframe()

,msg_id,name,content,context_level,function_name,call_id,usage,call_output,round
0,0,User,What is the weather like in Stuttgart Germany ...,0,NaN,NaN,NaN,NaN,0
1,1,Assistant,NaN,0,read_skill_instruction,call_UKds89nhPON608QVngFl2eE2,"{'input_tokens': 928, 'input_tokens_details': ...",NaN,0
2,2,System,NaN,0,NaN,call_UKds89nhPON608QVngFl2eE2,NaN,"""---\nname: web-search \ndescription: Guide...",0
3,3,Assistant,NaN,0,execute_skill_script,call_yxI3Fw3moNsIUG2sajVpjhzk,"{'input_tokens': 1881, 'input_tokens_details':...",NaN,0
4,4,System,NaN,0,NaN,call_yxI3Fw3moNsIUG2sajVpjhzk,NaN,"""Error executing tool execute_skill_script: Ag...",0
5,5,Assistant,NaN,0,execute_skill_script,call_5XDSbMYgjjA36Hk1m1tvNP1P,"{'input_tokens': 1949, 'input_tokens_details':...",NaN,0
6,6,System,NaN,0,NaN,call_5XDSbMYgjjA36Hk1m1tvNP1P,NaN,"""Error executing script: usage: web_search.py ...",0
7,7,Assistant,NaN,0,execute_skill_script,call_GBPZUpdkM4JFY8Yni1G6rpTS,"{'input_tokens': 2038, 'input_tokens_details':...",NaN,0
8,8,System,NaN,0,NaN,call_GBPZUpdkM4JFY8Yni1G6rpTS,NaN,"""[\n {\n \""title\"": \""Stuttgart - Wi...",0
9,9,Assistant,NaN,0,execute_skill_script,call_TWrsU1EXBb0KBz9M6077CiOM,"{'input_tokens': 2581, 'input_tokens_details':...",NaN,0
